In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_path = "/data/jupiter/datasets/mannequin_in_dust/"
df = pd.read_csv("/mnt/sandbox1/alex.li/summary.csv")
output_path = "/data/jupiter/li.yu/exps/driveable_terrain_model/dust_51_v188_58d_rak_local_fine_tversky11_sum_image_normT_prod5_airdyn_r3a8_s30/mannequin_in_dust_dustpred/"
df_dust = pd.read_csv(output_path + "/output.csv")
df = pd.merge(df, df_dust, on='id')
output_path = "/mnt/sandbox1/alex.li/results/mannequin_in_dust/7class_prod/results"

In [ ]:
print(df['pred_dust_ratio'].describe())
print((df['pred_dust_ratio'] > .10).mean())
# print((df_dust[''] > .15).mean())
# print((df_dust['total_thresholded_dust_ratio'] > .05).mean())

In [ ]:
df.columns

In [ ]:
sum(df['predicted_stop'])

In [ ]:
from dl.utils.io_utils import normalize_image
%matplotlib inline 
df_neg = df[df['predicted_stop'] == False]
def plot(ind):
    data = np.load(data_path + 'processed/images/' + df_neg.iloc[ind]['id'] + '/stereo_output.npz')
    left = normalize_image(data['left'], True)
    plt.imshow(left)
    plt.show()
    print(f"DUSTINESS: {df_neg.iloc[ind]['pred_dust_ratio']}")
plot(4)

In [ ]:
plot(29)

In [ ]:
plot(37)

In [ ]:
%matplotlib inline
plt.scatter(df['pred_dust_ratio'], df['predicted_stop'], s=.5)

In [ ]:
df['pred_dust_ratio']

In [ ]:
bins = np.arange(0, np.max(df['pred_dust_ratio'])+.01, 2.5)
plt.hist(df[(df['predicted_stop'] == True) & (df['label_stop'] == True)]['pred_dust_ratio'], bins=bins, alpha=.7, label='tp', color='blue')
plt.hist(df[(df['predicted_stop'] == False) & (df['label_stop'] == True)]['pred_dust_ratio'], bins=bins, alpha=.6, label='fn', color='orange')

In [ ]:
df['collected_on_dt'] = pd.to_datetime(df['collected_on'])

In [ ]:
plt.scatter(df['collected_on_dt'], df['pred_dust_ratio'], s=.5)
plt.xlabel("time")
plt.ylabel("dust level")

In [ ]:
df.sort_values('collected_on')['pred_dust_ratio']
diff = df.sort_values('collected_on')['pred_dust_ratio'].diff()[1:]

In [ ]:
print(diff[1:].std())

In [ ]:
plt.hist(df['pred_dust_ratio'])

# Dust threshold such that, for all images with dust below the threshold
TP / (TP + FN) >= .9

In [ ]:
# Li method
# TP + FN = len(df)
FN = TP = 0
df = df.sort_values('pred_dust_ratio')
for i in range(len(df)):
    row = df.iloc[i]
    if row['predicted_stop'] == False:
        FN += 1
    else:
        TP += 1
    x = TP / (TP + FN)
    if abs(x - .9) < .002:
        print(row['pred_dust_ratio']) # ~20

In [ ]:
import scipy

In [ ]:
df.columns

In [ ]:
# plt.show(df['collected_on'], df['pred_dust_ratio'])

In [ ]:
# Alex method
# TP + FN = len(df)
df = df.sort_values('pred_dust_ratio')
interval_size = 200
interval_step = 20
max_valid_ratio = 0
dust_ratio = []
tp_rate = []
human_tp_rate = []
for i in range(0, len(df) - interval_size, interval_step):
    interval = df.iloc[i:i+interval_size]
    TP = interval['predicted_stop'].sum()
    LABEL_P = interval['label_stop'].sum()
    FN = LABEL_P - TP
    r = interval.iloc[interval_size  // 2]['pred_dust_ratio']
    dust_ratio.append(r)
    tp_rate.append(TP / interval_size)
    human_tp_rate.append(interval['label_stop'].sum() / interval_size)
    # how confident are we that it will work on 90% of humans or more at this dust level?
    confidence = scipy.stats.binomtest(TP, LABEL_P, .9, 'less').pvalue
    # if confidence > .9:
    #     max_valid_ratio = r
    # Just check directly
    if TP / LABEL_P > .9:
        max_valid_ratio = r
%matplotlib inline 
plt.scatter(dust_ratio, tp_rate, label='model tp rate')
plt.scatter(dust_ratio, human_tp_rate, label='human labeler tp rate')
plt.axhline(.9,  linestyle='--')
plt.axvline(max_valid_ratio, linestyle='--')
plt.xlabel("Dust ratio (%)")
plt.ylabel("Human recall at this dust ratio")
plt.legend()
print(f"Recommended dust ratio: {max_valid_ratio:.1f}")

In [ ]:
# Carmine method
for fn_level in [.05, .1, .2, .3]:
    row = df_neg.sort_values('pred_dust_ratio').iloc[int(fn_level * len(df_neg))]
    print(row['Unnamed: 0'])
    print(f"{100 * fn_level}%: {row.pred_dust_ratio}")

In [ ]:
df_neg['pred_dust_ratio']

In [ ]:
sum(df['human_state'] == 'true_positive')